In [0]:
import numpy as np
import os
import string

import re

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, Reshape, LSTM, TimeDistributed
from keras.optimizers import Adam
from keras import callbacks
from keras.utils import to_categorical
from keras.models import load_model
import json
from keras.callbacks import *

Before start go to **Edit->Notebook setting->Hardware accelerator** and set GPU

In [0]:
def generator(features, labels, batch_size, symbols):
    # Create empty arrays to contain batch of features and labels#
    idx = np.arange(len(features))
    np.random.shuffle(idx)
    offset = 0
    while True:
        if offset + batch_size > len(idx):
            np.random.shuffle(idx)
            offset = 0

        # choose random index in features
        index = idx[offset: offset + batch_size]
        batch_features = to_categorical(
            features[index], num_classes=symbols)
        batch_labels = to_categorical(
            labels[index], num_classes=symbols)
        offset += batch_size
        yield batch_features, batch_labels

Suddenly, file with jokes is soooo big. We loaded it to google drive ant here we are to load it

In [0]:
#taken from this StackOverflow answer: https://stackoverflow.com/a/39225039
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

Getting acces to Google Drive

In [150]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Down here we load files from GD with links. It si realy easy to find part in link, wich is file_**id**

In [0]:
if not os.path.isdir("/content/gdrive/My Drive/BAneks"):
    os.mkdir("/content/gdrive/My Drive/BAneks")

file_id = '1B1XwxoVPjaLJVWKC4Fvet870VI4ohYMc'
destination = '/content/gdrive/My Drive/BAneks/BAneks.npy'
download_file_from_google_drive(file_id, destination)

In [0]:
if not os.path.isdir("/content/gdrive/My Drive/BAneks"):
    os.mkdir("/content/gdrive/My Drive/BAneks")

file_id = '1IecESTQFca14oEPhN6M3Aylae3zeJKL6'
destination = '/content/gdrive/My Drive/BAneks/Humoreski.npy'
download_file_from_google_drive(file_id, destination)

Let us open file

In [0]:
def losdList(filename):
    tempNumpyArray = np.load(filename)
    return tempNumpyArray.tolist()

In [0]:
jokes_b = losdList('/content/gdrive/My Drive/BAneks/BAneks.npy')
jokes_h = losdList('/content/gdrive/My Drive/BAneks/Humoreski.npy')

In [0]:
jokes = jokes_b + jokes_h

let's rm some long jokes

In [137]:
elems_to_rm = []
for i in range(len(jokes)):
  if 200 < len(jokes[i]):
    elems_to_rm.append(jokes[i])
for e in elems_to_rm:
  jokes.remove(e)
print(len(jokes))

5888


And pre-proc jokes


In [0]:
# rm punctuation
jokes = [j.translate(str.maketrans('', '', string.punctuation)) for j in jokes]
# lower case
jokes = [j.lower() for j in jokes]
# rm everything but proper(Rus) letters
letters = list([chr(i) for i in range(ord('а'),ord('а')+32)]) + ['ё', ' ']
new_jokes = []
for j in jokes:
  for char in j:
    if char not in letters:
      j = j.replace(char, '')
  new_jokes.append(j)
jokes = new_jokes

Creating maps char <-> number

In [139]:
everything = sorted(set(''.join(jokes)))
char_map = {}
reverse_char_map = {}
for i, c in enumerate(everything):
  char_map[c] = i + 2
  reverse_char_map[i + 2] = c
symbols = len(char_map) + 2
print('symbols: ', symbols)

symbols:  36


Creating validation and train

In [0]:
features = list(
    map(lambda j: np.array([1] + list(map(lambda c: char_map.get(c, char_map[' ']), j)) + [0], dtype='int8'),
        jokes))

largest = max(map(len, features))
x = np.zeros([len(features), largest], dtype='int8')
y = np.zeros([len(features), largest], dtype='int8')

for i, f in enumerate(features):
  x[i, :len(f)] = f
  y[i, :len(f) - 1] = f[1:]

idx = np.arange(len(x))
np.random.shuffle(idx)

validation_split = 0.2
val_samples = int(validation_split * len(x))

val_idx = idx[:val_samples]
val_x = to_categorical(x[val_idx], num_classes=symbols)
val_y = to_categorical(y[val_idx], num_classes=symbols)

train_idx = idx[val_samples:]
x = x[train_idx]
y = y[train_idx]

batch_size = 64

[Here](https://medium.com/@mukesh.kumar43585/model-checkpoint-google-colab-and-drive-as-persistent-storage-for-long-training-runs-e35ffa0c33d9)  I read how to save model in Colab. Someone says that colab often crash.

In [0]:
if not os.path.isdir("/content/gdrive/My Drive/BAneks/BAneksCheckPoints"):
    os.mkdir("/content/gdrive/My Drive/BAneks/BAneksCheckPoints")

In [0]:
filepath="/content/gdrive/My Drive/BAneks/BAneksCheckPoints/epochs:{epoch:03d}-val_acc:{categorical_accuracy:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='categorical_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

Firstly let's defind model

In [144]:
model = Sequential()
model.add(LSTM(1024, return_sequences=True, input_shape=[largest, symbols]))
model.add(Dropout(0.2))
model.add(LSTM(1024, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(1024, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(1024, return_sequences=True))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(symbols, activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['categorical_accuracy'])
print(model.summary())


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 195, 1024)         4345856   
_________________________________________________________________
dropout_21 (Dropout)         (None, 195, 1024)         0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 195, 1024)         8392704   
_________________________________________________________________
dropout_22 (Dropout)         (None, 195, 1024)         0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 195, 1024)         8392704   
_________________________________________________________________
dropout_23 (Dropout)         (None, 195, 1024)         0         
_________________________________________________________________
lstm_24 (LSTM)               (None, 195, 1024)        

And finaly let's train it


In [146]:
model.fit_generator(
    generator(x, y, batch_size, symbols), steps_per_epoch=len(x) // batch_size,
    epochs=30, shuffle=False, validation_data=(val_x  , val_y),
    callbacks=callbacks_list, verbose=1
    )
print("Whoho done")

Epoch 1/30
73/73 [==============================] - 127s 2s/step - loss: 1.5084 - categorical_accuracy: 0.6184 - val_loss: 1.3823 - val_categorical_accuracy: 0.6304

Epoch 00001: categorical_accuracy improved from -inf to 0.61837, saving model to /content/gdrive/My Drive/BAneks/BAneksCheckPoints/epochs:001-val_acc:0.618.hdf5
Epoch 2/30
73/73 [==============================] - 119s 2s/step - loss: 1.3467 - categorical_accuracy: 0.6387 - val_loss: 1.3463 - val_categorical_accuracy: 0.6346

Epoch 00002: categorical_accuracy improved from 0.61837 to 0.63873, saving model to /content/gdrive/My Drive/BAneks/BAneksCheckPoints/epochs:002-val_acc:0.639.hdf5
Epoch 3/30
73/73 [==============================] - 119s 2s/step - loss: 1.2991 - categorical_accuracy: 0.6453 - val_loss: 1.2898 - val_categorical_accuracy: 0.6451

Epoch 00003: categorical_accuracy improved from 0.63873 to 0.64533, saving model to /content/gdrive/My Drive/BAneks/BAneksCheckPoints/epochs:003-val_acc:0.645.hdf5
Epoch 4/30
73

Now let's load model


In [0]:
model = Sequential()
model.add(LSTM(1024, return_sequences=True, input_shape=[largest, symbols]))
model.add(Dropout(0.2))
model.add(LSTM(1024, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(1024, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(1024, return_sequences=True))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(symbols, activation='softmax')))
model.load_weights("/content/gdrive/My Drive/BAneks/BAneksCheckPoints/epochs:024-val_acc:0.839.hdf5")
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['categorical_accuracy'])

In [0]:
for j in range(100):
  gen = np.zeros([1, largest, symbols], dtype='int8')
  gen[0][0][1] = 1
  out = ''
  for i in range(1, 20):
    pred = model.predict(gen)
    pred = pred[0][i - 1]
    letter = np.random.choice(symbols, p=pred)
    if letter == 0 or letter == 1:
      break
    out += reverse_char_map[letter]
    gen[0][i][letter] = 1
  print(out)

 отец у тебя как ра
две невсестой людя 
как называется мужч
азтерская российски
один раз был вистир
у в созе банзом сол
сидят настолько нач
 е индарцаз на фере
четыре парнят был б
это панки глядя нас
почему плетенки хим
однажды эрнест хеми
ктэто придумаю линд
кмот следиен и расс
поддога лишь ваша ж
умер от неродой вид
 почему голосовы мо
сколько луба белых 
если спрашивует у а
если согласин стран
приходит дву полёву
в куритерсии просис
жена мужу куда а ты
приходит мужик к та
невмогла бы знает н
штирлиц андал дразп
сидят не бояться в 
аёокосек оненьки уч
темпида тетрадена с
сколько нужно красп
сказала пренесута р
 почему никто не хо

 сынок ныплохо себя
на словам девушки в
 звенит какая рука 
напяйщего остановки
елимистужросит заяц
